# Морфология 2
В данном ноутбуке находится задание на триграммы и словарь. В качестве текста будем использовать томик войны и мира: https://goo.gl/qcVTLE 

Для начала напомним некоторые полезные инструменты на питоне

## 1. Инструменты

### 1.1. Регулярные выражения
Исчерпывающий пост https://habr.com/ru/post/349860/

In [1]:
import re

# С помощью рег. выражения можно искать, заменять и сентезировать строки по шаблонам
# Парочка простых примеров
numbers = re.findall(r'\d+', r'There is some numbers: 49 and 432')
print(u'Находим числа в строке: ', numbers)

print(u'Простенький токенизатор: ', re.sub('[,\.?!]',' ','How, to? split. text!').split())

print(u'Еще один токенизатор: ', re.split(r'\W+', 'How, to? split. text! Again'))

# в качестве тренировки придумайте свой токенизатор в случае, когда из текста нужно получить только русские слова.

Находим числа в строке:  ['49', '432']
Простенький токенизатор:  ['How', 'to', 'split', 'text']
Еще один токенизатор:  ['How', 'to', 'split', 'text', 'Again']


### 1.2. Чтение файлов
Чтобы не мучится самим с кодировками, приведем способ чтения файла корпуса.

In [2]:
import io
import razdel
wordlist = None

with io.open('wp.txt', "r") as text_file:
    text = text_file.read()
    wordlist = razdel.tokenize(text.lower())
    # wordlist = Здесь заиспользуем токенизатор. Также приведем все слова к нижнему регистру.

FileNotFoundError: [Errno 2] No such file or directory: 'wp.txt'

### 1.3. ООП
В питоне можно создавать свои классы, скорее всего нам понадобится класс для хранения бора

In [ ]:
class Trie:
    def __init__(self):
        self.children = {}
        self.is_word = False
    
    def add(self, word):
        if not word:
            print('Аргумент не является словом')
            
        head, *tail = word
        
        if head not in self.children:
            self.children[head] = Trie()
        
        node = self.children[head]
        
        if tail:
            node.add(tail)
        else:
            self.is_word = True

In [ ]:
trie = Trie()
trie.add('he')
trie.add('she')
trie.add('his')
trie.add('hers')
trie.children

### 1.4. pyplot
Довольно часто приходится построить какие-нибудт графики и гистограммы для изучения данных, с которыми придется работать. Для этого в питоне есть удобнейший модуль для этого - pyplot. Туториал: https://matplotlib.org/3.1.1/tutorials/introductory/pyplot.html

In [ ]:
#!pip install matplotlib # Установка модуля
import matplotlib.pyplot as plt
# команда чтобы картинки рисовались прямо в ноутбуке
%matplotlib inline 

In [ ]:
# Посмотрим на график функции x^2
data = [i * i for i in range(100)]
plt.plot(data)

In [ ]:
# и распределение по значениям
plt.hist(data)

Также полезная команда для подсчета времения выполнеия в ячейке

In [3]:
%%time
res = 0
for i in range(int(1e8)):
    res += 1

Wall time: 10.5 s


## 2. Задание
Собственно теперь нам потребуется написать подсчет триграмм и бор для словаря, чтобы реализовать простенькую систему исправления ошибок.

### 2.1. Триграммы
Для начала получим словарь триграмма - её кол-во в тексте, не забывая о начале и конце слова.

Далее можно построить гистограмму на кол-во вхождений триграмм, и посмотреть самые частые и какая частота у каких-нибудь заведомо странных, например: 'чгк', 'шкя' и подобные. Из полученных данных придумать эвристику, как считать слово с опечаткой. Проверить эвристику по словарю.

In [4]:
import io
import razdel
wordlist = None

with io.open('war_and_piece.txt', "r") as text_file:
    text = text_file.read()
    wordlist = list(razdel.tokenize(text.lower()))

In [5]:
def build_corpus_trigrams(words):
    trigrams_and_freq = {}
    
    for word in words:
        marked_word = '##' + word.text + '##'
        
        for char in range(len(marked_word) - 2):
            trigram = marked_word[char: char + 3]
            
            if trigram in trigrams_and_freq:
                trigrams_and_freq[trigram] += 1
            else:
                trigrams_and_freq[trigram] = 1
    
    return trigrams_and_freq

        
def is_error(word, trigrams_and_freq):
    marked_word = '##' + word.text + '##' 
    
    errors = []
    # будем учитывать две триграммы
    for char in range(1, len(marked_word) - 2):
        first_trigram = marked_word[char - 1: char + 2]
        second_trigram = marked_word[char: char + 3]
        votes = 0
        
        if first_trigram in trigrams_and_freq and trigrams_and_freq[first_trigram] > 2:
            votes += 0
        else:
            votes += 1
        
        if second_trigram in trigrams_and_freq and trigrams_and_freq[second_trigram] > 2:
            votes += 0
        else:
            votes += 1
            
        if votes == 2: 
            errors.append(first_trigram)
            errors.append(second_trigram)
            
    if errors:
        return set(errors)
    else:
        return False

In [6]:
def print_errors(words):
    trigram_dict = build_corpus_trigrams(words)
    for word in words:
        result = is_error(word, trigram_dict)
        if result:
            print('Cлово ->', word.text, '| Возможные ошибки ->', result)

In [7]:
print_errors(wordlist)

Cлово -> lucques | Возможные ошибки -> {'ucq', 'luc'}
Cлово -> atrocites | Возможные ошибки -> {'roc', 'oci'}
Cлово -> генуа | Возможные ошибки -> {'нуа', 'уа#'}
Cлово -> лукка | Возможные ошибки -> {'кка', 'укк'}
Cлово -> июле | Возможные ошибки -> {'#ию', 'июл', 'юле'}
Cлово -> virulente | Возможные ошибки -> {'vir', 'iru'}
Cлово -> insipides | Возможные ошибки -> {'sip', 'ipi'}
Cлово -> a-t-on | Возможные ошибки -> {'t-o', '-on'}
Cлово -> novosiizoff | Возможные ошибки -> {'iiz', 'sii', 'nov', 'izo', 'ovo'}
Cлово -> a-t-on | Возможные ошибки -> {'t-o', '-on'}
Cлово -> отжившим | Возможные ошибки -> {'отж', 'тжи'}
Cлово -> гидру | Возможные ошибки -> {'идр', 'гид'}
Cлово -> самоотвержения | Возможные ошибки -> {'оот', 'моо'}
Cлово -> montmorency | Возможные ошибки -> {'ncy', 'cy#', 'ntm', 'tmo'}
Cлово -> rohans | Возможные ошибки -> {'oha', 'roh'}
Cлово -> монморанси | Возможные ошибки -> {'нмо', 'онм'}
Cлово -> abbe | Возможные ошибки -> {'abb', 'bbe'}
Cлово -> императрица-мать | Во

Cлово -> туалетах | Возможные ошибки -> {'туа', 'уал'}
Cлово -> жемчугах | Возможные ошибки -> {'емч', 'чуг', 'мчу'}
Cлово -> толще | Возможные ошибки -> {'лще', 'олщ'}
Cлово -> conjure | Возможные ошибки -> {'nju', 'jur'}
Cлово -> soutndra | Возможные ошибки -> {'tnd', 'utn'}
Cлово -> avant-hier | Возможные ошибки -> {'t-h', '-hi'}
Cлово -> мадмуазель | Возможные ошибки -> {'адм', 'дму'}
Cлово -> алгебры | Возможные ошибки -> {'алг', 'лге', 'геб'}
Cлово -> ногтем | Возможные ошибки -> {'огт', 'гте'}
Cлово -> элоизы | Возможные ошибки -> {'лои', 'эло'}
Cлово -> старчески-едким | Возможные ошибки -> {'и-е', '-ед'}
Cлово -> abc | Возможные ошибки -> {'bc#', 'abc'}
Cлово -> стерпится-слюбится | Возможные ошибки -> {'я-с', 'ся-'}
Cлово -> элоиза | Возможные ошибки -> {'лои', 'эло'}
Cлово -> effrayante | Возможные ошибки -> {'yan', 'aya'}
Cлово -> indissolubles | Возможные ошибки -> {'olu', 'lub'}
Cлово -> revolte | Возможные ошибки -> {'vol', 'olt'}
Cлово -> puis-je | Возможные ошибки -> {

Cлово -> ламбахе | Возможные ошибки -> {'мба', 'амб'}
Cлово -> амштетене | Возможные ошибки -> {'мшт', 'амш'}
Cлово -> 28-го | Возможные ошибки -> {'8-г', '28-'}
Cлово -> 30-го | Возможные ошибки -> {'30-', '#30'}
Cлово -> ямщику | Возможные ошибки -> {'ямщ', 'мщи', '#ям'}
Cлово -> hochgeboren | Возможные ошибки -> {'chg', 'hge', 'geb'}
Cлово -> генерала-фельдмаршала | Возможные ошибки -> {'а-ф', '-фе', 'ла-'}
Cлово -> циркуляр | Возможные ошибки -> {'ирк', '#ци', 'цир'}
Cлово -> bilibine | Возможные ошибки -> {'ibi', 'lib'}
Cлово -> colportaient | Возможные ошибки -> {'lpo', 'olp'}
Cлово -> professe | Возможные ошибки -> {'ofe', 'rof', 'fes'}
Cлово -> victoire | Возможные ошибки -> {'cto', 'ict'}
Cлово -> victorieuses | Возможные ошибки -> {'cto', 'ict'}
Cлово -> liebchen | Возможные ошибки -> {'ebc', 'bch', 'ieb'}
Cлово -> врбна | Возможные ошибки -> {'рбн', 'врб'}
Cлово -> лихтенфельс | Возможные ошибки -> {'нфе', 'енф'}
Cлово -> echauffouree | Возможные ошибки -> {'fou', 'ffo'}
Cло

Cлово -> chef | Возможные ошибки -> {'hef', 'ef#'}
Cлово -> пристально-холодным | Возможные ошибки -> {'-хо', 'о-х'}
Cлово -> 16 | Возможные ошибки -> {'16#', '#16'}
Cлово -> алё | Возможные ошибки -> {'лё#', 'алё'}
Cлово -> алё | Возможные ошибки -> {'лё#', 'алё'}
Cлово -> huzards | Возможные ошибки -> {'zar', 'huz', 'uza'}
Cлово -> pavlograd | Возможные ошибки -> {'log', 'rad', 'vlo', 'ad#', 'avl'}
Cлово -> huzards | Возможные ошибки -> {'zar', 'huz', 'uza'}
Cлово -> pavlograd | Возможные ошибки -> {'log', 'rad', 'vlo', 'ad#', 'avl'}
Cлово -> приученная | Возможные ошибки -> {'риу', 'иуч'}
Cлово -> государя-императора | Возможные ошибки -> {'я-и', 'ря-'}
Cлово -> государя-императора | Возможные ошибки -> {'я-и', 'ря-'}
Cлово -> 17-го | Возможные ошибки -> {'7-г', '#17', '17-'}
Cлово -> 19-го | Возможные ошибки -> {'9-г', '19-'}
Cлово -> 80-титысячная | Возможные ошибки -> {'80-', '#80'}
Cлово -> циферблате | Возможные ошибки -> {'циф', '#ци'}
Cлово -> обер-гофмаршалу | Возможные ошиб

### 2.2. Бор
Далее построим бор, с помощью которого будем искать исправления опечатки в слове. Пока только в случае замены и удаления в конце (Если есть желание, то можно и для всех случаев). Также сравнить по времени с поиском в случае замены в исходном слове последних двух символов и поиске в словаре.

In [8]:
class TrieNode():
    def __init__(self, char):
        self.char = char
        self.children = {}
        self.end = False
        
class Trie():
    def __init__(self):
        self.root = TrieNode("")
        
    def insert(self, word):
        node = self.root
        
        for char in word:
            if char in node.children:
                node = node.children[char]
            else:
                trie_node = TrieNode(char)
                node.children[char] = trie_node
                node = trie_node
                
        node.end = True
    
    def search(self, node, prefix):
        if node.end:
            self.output.append((prefix + node.char))
            
        for child in node.children.values():
            self.search(child, prefix + node.char)
            
    def get_words(self, prefix):
        self.output = []
        node = self.root
        
        for char in prefix:
            if char in node.children:
                node = node.children[char]
            else:
                return []
            
        self.search(node, prefix[:-1])

        return self.output
    
    def get_simillar_words(self, prefix, errors):
        pass

In [9]:
trie = Trie()

for word in wordlist:
    trie.insert(word.text)

In [15]:
def call_counter(func):
    def helper(*args, **kwargs):
        helper.calls += 1
        return func(*args, **kwargs)
    helper.calls = 0
    helper.__name__= func.__name__

    return helper


memo = {}
@call_counter
def levenshtein(s, t):
    if s == "":
        return len(t)
    if t == "":
        return len(s)
    cost = 0 if s[-1] == t[-1] else 1
       
    i1 = (s[:-1], t)
    if not i1 in memo:
        memo[i1] = levenshtein(*i1)
    i2 = (s, t[:-1])
    if not i2 in memo:
        memo[i2] = levenshtein(*i2)
    i3 = (s[:-1], t[:-1])
    if not i3 in memo:
        memo[i3] = levenshtein(*i3)
    res = min([memo[i1]+1, memo[i2]+1, memo[i3]+cost])
    
    return res

print(levenshtein("каровя", "корова"))

2


In [16]:
def correct_word(word):
    return trie.get_words(word)
    

def char_range(c1, c2):
    """Generates the characters from `c1` to `c2`, inclusive."""
    for c in range(ord(c1), ord(c2)+1):
        yield chr(c)

def correct_word_naive(word, dictionary):
    """
        Corrects word using naive approach (search most similar word in the dicitionary)
    """
    distances = [(candidate.text, levenshtein(word.lower(), candidate.text)) for candidate in dictionary]
    min_distance = sorted(distances, key=lambda x: x[1])[0][1]

    candidates = [word[0] for word in distances if word[1] == min_distance]

    return '<{}>: [{}]'.format(word, '|'.join(candidates))

In [17]:
correct_word_naive('коровя', wordlist)

'<коровя>: [короля|короля|корова]'

In [21]:
sample = '''лев николаевич талстой война и мип том чусть пегвая е паместья мой верный рап ну что князь генуа и лука стали не
бальше как паместьяни фамили бонапарте нет я вас предупреждааю если вы мне ни скажете что у нас вайна если вы еще позволете 
себе зашишать все гадасты все ужаси этого антихриста права я верю что он антехрест я вас больше не знаю вы уж не друх мой вы 
уш не мой верный раб как вы говориде ну здравствуйде здравствуйте я вижу что я вас пугаю садитесь и расказывайте так гаворила
в июле года известная анна павловна шерер фрейлина и'''

In [22]:
%%time

correct_word_naive(wordlist, sample)

KeyboardInterrupt: 

### 2.3 Все вместе
Теперь соберем поиск и исправление опечаток в одну систему, которая будет принимать текст и править его. Также замерим скорость по сравнению с "менее наивным подходом".

In [ ]:
def correct_mistakes(text):
    '''returns corrected text'''
    pass

def correct_mistakes_naive(text):
    '''returns corrected text using generation'''
    pass